In [138]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests 
import re
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm.auto import tqdm


headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

In [139]:
# service = Service("/Users/panavshah/Desktop/Desktop - Panav’s MacBook Pro/chromedriver-mac-x64/chromedriver")
# driver = webdriver.Chrome(service=service)

# url = "https://www.tennisabstract.com/cgi-bin/leaders.cgi?f="

# driver.get(url)
# driver.implicitly_wait(10)
# html = driver.page_source
# print(html)

# with open("players.html", "w") as f:
#     f.write(html)


In [140]:
with open("players.html") as file:
    soup = BeautifulSoup(file, "html.parser")
table_players = soup.find(class_='tablesorter')
players_a = table_players.find_all('a')
players = []
ctr = 0
for player_a in players_a:
    if ctr % 2 == 0:
        players.append(player_a.get_text().replace(" ", ""))
    ctr += 1

In [141]:
# players

In [142]:
# def get_webscrape_player_data(i, player):
#     url = f"https://www.tennisabstract.com/cgi-bin/player-classic.cgi?p={player}&f=ACareerqq"
#     r = requests.get(url, headers=headers)

#     soup = BeautifulSoup(r.content, 'html.parser')

#     with open(f"players_data/{i}_{player}.html", "w") as file:
#         file.write(str(soup))

#     # print(script_tag)

In [143]:
# for i, player in enumerate(players):
#     get_webscrape_player_data(i, player)

In [144]:
def get_player_match_data(i):
    with open(f"players_data/{i}_{players[i]}.html") as file:
        soup = BeautifulSoup(file, 'html.parser')
    scripts = soup.find_all('script')

    matchmx = None
    for script in scripts:
        if 'var matchmx' in script.text:
            matchmx_pattern = re.search(r'var matchmx\s*=\s*(\[[\s\S]*?\]);', script.text)
            if matchmx_pattern:
                matchmx = matchmx_pattern.group(1)
                break

    matchmx = json.loads(matchmx)

    # if matchmx:
    #     print("Extracted matchmx list:")
    #     print(len(matchmx))
    # else:
    #     print("matchmx variable not found.")

    Rk = []
    vRk = []
    score = []
    for match in matchmx:
        try:
            Rk_ = int(match[5])
            vRk_ = int(match[12])
            score_ = 0 if match[4] == 'L' else 1
            Rk.append(Rk_)
            vRk.append(vRk_)
            score.append(score_)
        except:
            continue
    
    return Rk, vRk, score

In [145]:
Rk_total, vRk_total, score_total = [], [], []

In [146]:
# Rk, vRk, score = get_player_match_data(5)
# df = pd.DataFrame({'Rk': Rk, 'vRk': vRk, 'score': score})
# df.head()

for i in range(0, 50):
    Rk, vRk, score = get_player_match_data(i)
    Rk_total.extend(Rk)
    vRk_total.extend(vRk)
    score_total.extend(score)

df = pd.DataFrame({'Rk': Rk_total, 'vRk': vRk_total, 'score': score_total})
df.dropna(inplace=True)

In [147]:
df.tail()

,Rk,vRk,score
26246,1567,818,0
26247,1567,1279,1
26248,1567,1332,1
26249,1567,929,1
26250,1563,658,0


In [148]:
X = df[['Rk', 'vRk']]
y = df['score']
X.shape, y.shape

((26251, 2), (26251,))

In [149]:

class RankComparisonModel(nn.Module):
    def __init__(self):
        super(RankComparisonModel, self).__init__()
        self.fc1 = nn.Linear(2, 64)  # Two inputs (ranks)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)  # Output a single percentage value
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()  # To output probability (0-1)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

model0 = RankComparisonModel()

In [150]:
type(X.to_numpy())

numpy.ndarray

In [151]:
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(), y.to_numpy(), test_size=0.2, random_state=42)
X_train = torch.from_numpy(X_train.astype(np.float32))  # Ensure it's in the right dtype for torch
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([21000, 2]),
 torch.Size([5251, 2]),
 torch.Size([21000]),
 torch.Size([5251]))

In [152]:
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_dataloader = DataLoader(train_dataset, batch_size=256, shuffle=True)  # Adjust batch_size as needed
test_dataloader = DataLoader(test_dataset, batch_size=256, shuffle=False)  # No shuffling for test data
print(len(train_dataloader), len(test_dataloader))

83 21


In [153]:
X_batch, y_batch = next(iter(train_dataloader))
model0(X_batch)[:10]

tensor([[8.3843e-09],
        [1.1177e-16],
        [1.6130e-11],
        [2.8387e-27],
        [4.4404e-09],
        [0.0000e+00],
        [8.0112e-12],
        [1.9418e-30],
        [2.8700e-09],
        [5.2009e-10]], grad_fn=<SliceBackward0>)

In [154]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,):
    
    model.train()

    train_loss, train_acc = 0, 0

    for batch, (X, y) in enumerate(dataloader):
        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epsilon = 1e-8
        train_acc += (np.abs(y_pred.detach() - y.detach())/(y.detach() + epsilon)).sum().item() / len(y_pred)
    
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc


In [155]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,):
    
    model.eval()

    test_loss, test_acc = 0, 0

    with torch.inference_mode():
        for batch, (X, y) in enumerate(dataloader):
            y_pred = model(X)
            loss = loss_fn(y_pred, y)
            test_loss += loss.item()
            epsilon = 1e-8
            test_acc += (np.abs(y_pred.detach() - y.detach())/(y.detach() + epsilon)).sum().item() / len(y_pred)
        
        test_loss += test_loss / len(dataloader)
        test_acc += test_acc / len(dataloader)
        return test_loss, test_acc 


In [156]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int = 5,):
    
    results = {"train_loss": [],
                "train_acc": [],
                "test_loss": [],
                "test_acc": []}
    
    for epoch in range(epochs):
        train_loss, train_acc = train_step(model=model,
                                           dataloader=train_dataloader,
                                           loss_fn=loss_fn,
                                           optimizer=optimizer,)
        test_loss, test_acc = test_step(model=model,
                                        dataloader=test_dataloader,
                                        loss_fn=loss_fn,)

        print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Test loss: {test_loss:4f} | Test acc: {test_acc:.4f}")
        results["train_loss"].append(train_loss)
        results["train_acc"].append(train_acc)
        results["test_loss"].append(test_loss)
        results["test_acc"].append(test_acc)       

    return results               


In [158]:
torch.manual_seed(42)

NUM_EPOCHS = 50

model_0 = RankComparisonModel()

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(params=model_0.parameters(),
                             lr=0.001)

model_0_results = train(model=model_0,
                        train_dataloader=train_dataloader,
                        test_dataloader=test_dataloader,
                        optimizer=optimizer,
                        loss_fn=loss_fn,
                        epochs=NUM_EPOCHS)
                        

Epoch: 0 | Train loss: 0.3339 | Train acc: 6614709424.0000 | Test loss: 6.047122 | Test acc: 153088748750.5867
Epoch: 1 | Train loss: 0.2600 | Train acc: 6121593009.4458 | Test loss: 6.540978 | Test acc: 168757049788.8128
Epoch: 2 | Train loss: 0.2962 | Train acc: 7388194492.6265 | Test loss: 6.055964 | Test acc: 155096733723.5449
Epoch: 3 | Train loss: 0.2697 | Train acc: 6461337111.3253 | Test loss: 5.627082 | Test acc: 141922474346.5504
Epoch: 4 | Train loss: 0.2585 | Train acc: 6237976282.9880 | Test loss: 6.037461 | Test acc: 157769965197.2606
Epoch: 5 | Train loss: 0.2507 | Train acc: 6021421689.8313 | Test loss: 5.328009 | Test acc: 132398933992.5496
Epoch: 6 | Train loss: 0.2503 | Train acc: 6051827562.6024 | Test loss: 5.757585 | Test acc: 150534275979.1203
Epoch: 7 | Train loss: 0.2504 | Train acc: 6143794101.2048 | Test loss: 6.292543 | Test acc: 92721778421.8568
Epoch: 8 | Train loss: 0.2526 | Train acc: 6167122880.0000 | Test loss: 5.438475 | Test acc: 116183516402.6928
Ep

In [169]:
model_0.eval()
with torch.inference_mode():
    print(model_0(torch.tensor([40, 1], dtype=torch.float32).unsqueeze(0)))

tensor([[0.6221]])


In [172]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logreg = LogisticRegression()
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
accuracy_score(y_test, y_pred)


0.6396876785374215